## Agent

This notebook demonstrates how to use Cua's Agent to run workflows in virtual sandboxes, either using Cua Cloud Containers or local VMs on Apple Silicon Macs.

### Installation

In [ ]:
!pip uninstall -y cua-agent

In [ ]:
!pip install "cua-agent[all]"

# Or install individual agent loops:
# !pip install cua-agent[openai]
# !pip install cua-agent[anthropic]
# !pip install cua-agent[uitars]
# !pip install cua-agent[omni]

In [ ]:
# If locally installed, use this instead:
import os

os.chdir('../libs/python/agent')
!poetry install
!poetry build

!pip uninstall cua-agent -y
!pip install ./dist/cua_agent-0.1.0-py3-none-any.whl --force-reinstall

## Initialize a Computer Agent

Agent allows you to run an agentic workflow in virtual sandbox instances. You can choose between cloud containers or local VMs.

In [ ]:
from computer import Computer, VMProviderType
from agent import ComputerAgent, LLM, AgentLoop, LLMProvider

In [ ]:
import os

# Get API keys from environment or prompt user
anthropic_key = os.getenv("ANTHROPIC_API_KEY") or \
    input("Enter your Anthropic API key: ")
openai_key = os.getenv("OPENAI_API_KEY") or \
    input("Enter your OpenAI API key: ")

os.environ["ANTHROPIC_API_KEY"] = anthropic_key
os.environ["OPENAI_API_KEY"] = openai_key

## Option 1: Agent with Cua Cloud Containers

Use cloud containers for running agents from any system without local setup.

### Prerequisites for Cloud Containers

To use Cua Cloud Containers, you need to:
1. Sign up at https://trycua.com
2. Create a Cloud Container
3. Generate an API Key

Once you have these, you can connect to your cloud container and run agents on it.

Get Cua API credentials and container details

In [ ]:
cua_api_key = os.getenv("CUA_API_KEY") or \
    input("Enter your Cua API Key: ")
container_name = os.getenv("CONTAINER_NAME") or \
    input("Enter your Cloud Container name: ")

Choose the OS type for your container (linux or macos)

In [ ]:
os_type = input("Enter the OS type of your container (linux/macos) [default: linux]: ").lower() or "linux"

### Create an agent with cloud container

In [ ]:
import logging
from pathlib import Path

# Connect to your existing cloud container
computer = Computer(
    os_type=os_type,
    api_key=cua_api_key,
    name=container_name,
    provider_type=VMProviderType.CLOUD,
    verbosity=logging.INFO
)

# Create agent
agent = ComputerAgent(
    computer=computer,
    loop=AgentLoop.OPENAI,
    model=LLM(provider=LLMProvider.OPENAI),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    only_n_most_recent_images=3,
    verbosity=logging.INFO
)


Run tasks on cloud container

In [ ]:
tasks = [
    "Open a web browser and navigate to GitHub",
    "Search for the trycua/cua repository",
    "Take a screenshot of the repository page"
]

for i, task in enumerate(tasks):
    print(f"\nExecuting task {i+1}/{len(tasks)}: {task}")
    async for result in cloud_agent.run(task):
        # print(result)
        pass
    print(f"✅ Task {i+1}/{len(tasks)} completed: {task}")


## Option 2: Agent with Local VMs (Lume daemon)

For Apple Silicon Macs, run agents on local VMs with near-native performance.

Before we can create an agent, we need to initialize a local computer with Lume.

In [ ]:
import logging
from pathlib import Path


computer = Computer(
    verbosity=logging.INFO, 
    provider_type=VMProviderType.LUME,
    display="1024x768",
    memory="8GB",
    cpu="4",
    os_type="macos"
)

### Create an agent with local VM

Let's start by creating an agent that relies on the OpenAI API computer-use-preview model.

In [ ]:
# Create agent with Anthropic loop and provider
agent = ComputerAgent(
    computer=computer,
    loop=AgentLoop.OPENAI,
    model=LLM(provider=LLMProvider.OPENAI),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    only_n_most_recent_images=3,
    verbosity=logging.INFO
)

Run tasks on a local Lume VM

In [ ]:
tasks = [
    "Look for a repository named trycua/cua on GitHub.",
    "Check the open issues, open the most recent one and read it.",
    "Clone the repository in users/lume/projects if it doesn't exist yet.",
    "Open the repository with an app named Cursor (on the dock, black background and white cube icon).",
    "From Cursor, open Composer if not already open.",
    "Focus on the Composer text area, then write and submit a task to help resolve the GitHub issue.",
]

for i, task in enumerate(tasks):
    print(f"\nExecuting task {i}/{len(tasks)}: {task}")
    async for result in agent.run(task):
        # print(result)
        pass

    print(f"\n✅ Task {i+1}/{len(tasks)} completed: {task}")

Or using the Omni Agent Loop:

In [ ]:
import logging
from pathlib import Path
from agent import ComputerAgent, LLM, AgentLoop

computer = Computer(verbosity=logging.INFO)

# Create agent with Anthropic loop and provider
agent = ComputerAgent(
        computer=computer,
        loop=AgentLoop.OMNI,
        # model=LLM(provider=LLMProvider.ANTHROPIC, name="claude-3-7-sonnet-20250219"),
        # model=LLM(provider=LLMProvider.OPENAI, name="gpt-4.5-preview"),
        model=LLM(provider=LLMProvider.OLLAMA, name="gemma3:12b-it-q4_K_M"),
        save_trajectory=True,
        trajectory_dir=str(Path("trajectories")),
        only_n_most_recent_images=3,
        verbosity=logging.INFO
    )

tasks = [
    "Look for a repository named trycua/cua on GitHub.",
    "Check the open issues, open the most recent one and read it.",
    "Clone the repository in users/lume/projects if it doesn't exist yet.",
    "Open the repository with an app named Cursor (on the dock, black background and white cube icon).",
    "From Cursor, open Composer if not already open.",
    "Focus on the Composer text area, then write and submit a task to help resolve the GitHub issue.",
]

for i, task in enumerate(tasks):
    print(f"\nExecuting task {i}/{len(tasks)}: {task}")
    async for result in agent.run(task):
        # print(result)
        pass

    print(f"\n✅ Task {i+1}/{len(tasks)} completed: {task}")

## Using the Gradio UI

The agent includes a Gradio-based user interface for easy interaction. To use it:

In [ ]:
from agent.ui.gradio.app import create_gradio_ui

app = create_gradio_ui()
app.launch(share=False)

## Advanced Agent Configurations

### Using different agent loops

You can use different agent loops depending on your needs:

1. OpenAI Agent Loop

In [ ]:
openai_agent = ComputerAgent(
    computer=computer,  # Can be cloud or local
    loop=AgentLoop.OPENAI,
    model=LLM(provider=LLMProvider.OPENAI),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    verbosity=logging.INFO
)


2. Anthropic Agent Loop

In [ ]:
anthropic_agent = ComputerAgent(
    computer=computer,
    loop=AgentLoop.ANTHROPIC,
    model=LLM(provider=LLMProvider.ANTHROPIC),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    verbosity=logging.INFO
)


3. Omni Agent Loop (supports multiple providers)

In [ ]:
omni_agent = ComputerAgent(
    computer=computer,
    loop=AgentLoop.OMNI,
    model=LLM(provider=LLMProvider.ANTHROPIC, name="claude-3-7-sonnet-20250219"),
    # model=LLM(provider=LLMProvider.OPENAI, name="gpt-4.5-preview"),
    # model=LLM(provider=LLMProvider.OLLAMA, name="gemma3:12b-it-q4_K_M"),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    only_n_most_recent_images=3,
    verbosity=logging.INFO
)


4. UITARS Agent Loop (for local inference on Apple Silicon)

In [ ]:
uitars_agent = ComputerAgent(
    computer=computer,
    loop=AgentLoop.UITARS,
    model=LLM(provider=LLMProvider.UITARS),
    save_trajectory=True,
    trajectory_dir=str(Path("trajectories")),
    verbosity=logging.INFO
)


### Trajectory viewing

All agent runs save trajectories that can be viewed at https://trycua.com/trajectory-viewer

In [ ]:
print(f"Trajectories saved to: {Path('trajectories').absolute()}")
print("Upload trajectory files to https://trycua.com/trajectory-viewer to visualize agent actions")
